## Análisis del proceso Halving y su efecto en el precio del bitcoin

# Extracción de datos

In [2]:
####input your credentials here
consumer_key = 
consumer_secret = 
access_token = 
access_token_secret = 

In [3]:
import tweepy

In [4]:
# Autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Módulo api de Tweepy y con wait lo que hacemos es que si se acaba el cupo de redlimits mi programa no se cae
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True) 


In [6]:
# Hagamos una búsqueda de Tweets con la palabra #bitcoin AND #halving price

for tweet in tweepy.Cursor(api.search,q=('#bitcoin', '#halving', 'price'),count=100,
                           tweet_mode= 'extended').items(10):
    print(tweet._json['full_text'])
    
    if 'retweeted_status' in tweet._json:
        print("Es un RT y su texto completo es: " + tweet._json['retweeted_status']['full_text'])



1 |
- Bitcoin #hashrate has dropped by almost a third since #halving 
- #Satoshi may be back 
- #Bitcoin price is falling 
- permanent and future #scalability problems 
- #centralization problem 
- overpriced #transactionfees 
- no #usecases
#Bitcoin Users Are Paying $7 #USD in Fees Per Transaction After It Spikes 265% Post-#Halving

Current #BTC Price: 📉 $9,356.63 (-4.23%)

https://t.co/4hmzkPGXUr
#Miner revenues crash, but this has always marked #bitcoin price bottoms https://t.co/vnnXIBjlpW $BTC #crypto #cryptocurrency #macro #halving #bullish #trend #volatility #PuellMultiple #RSI #MACD
RT @SmartCryptoNew1: MARKET VIEWS📈
#Bitcoin (BTC) options markets are finally taking off at CME. As #halving day uncertainties became less…
Es un RT y su texto completo es: MARKET VIEWS📈
#Bitcoin (BTC) options markets are finally taking off at CME. As #halving day uncertainties became less of a risk, institutional investors began mounting bullish positions.
DETAILS➡ https://t.co/mCy9lXALs1
#BTC #mar

# Guardado de datos en la base de datos MongoDb

In [39]:
# Importamos PyMongo
from pymongo import MongoClient

# Creamos una conexión con la BBDD
client = MongoClient()
           
# Usamos una base de datos llamada pec2
db = client.pec2

# Y cada Tweet es almacenado en una colección llamada "tweets"
for tweet in tweepy.Cursor(api.search,q=('#bitcoin', '#halving', 'price'),count=100,
                           tweet_mode= 'extended').items(5000):
    db.tweets.insert_one(tweet._json)

# Recuperación de datos y montaje del grafo

In [40]:
import networkx as nx

G = nx.DiGraph()

for result in db.tweets.find():
            uid = result['user']['screen_name']
            G.add_node(uid) # si ya existe lo omite solo
            
           
            if 'retweeted_status' in result:
                if G.has_edge(uid, result['retweeted_status']['user']['screen_name']):
                    G[uid][result['retweeted_status']['user']['screen_name']]['weight'] += 1.0
                else:
                    G.add_edge(uid, result['retweeted_status']['user']['screen_name'], weight = 1.0) 

In [41]:
len(list(G.node))


1989

Pasamos el grafico al formato graphml

In [42]:
#Guardamos en un graphml para después importarlo en Gephi

nx.write_graphml_lxml(G, "halving.graphml")